## Trabalho Prático - Banco de dados não relacionais

##### Aluno: Eduardo Augusto Nascimento Lima
##### Matrícula: 71727

### Enunciado do trabalho


##### Coletar informações de rede sociais ou importar dados externos e armazenar 1M de dados em um banco NoSQL.

##### Extrair informações do tipo:

##### - Termos mais frequentes;

#####  - Volume x dia;

##### - Volume x hora do dia

### Sofwtares aplicados

##### - Ambiente de desenvolvimento integrado (IDE) ==> Jupyter Notebook 3.5 (python);

##### - Banco de dados não relacionais (NoSQL) ==> Mongodb 3.2.10;

##### - Pacotes requeridos ==> pip;

##### - Comandos requeridos ==> pip install tweepy==3.3.0 / pip install pymongo

##### - módulos python ==>  tweepy, Datetime e Json;

### Palavras chaves para pesquisa

##### Foram escolhidos as seguintes palavras chaves, a partir do assunto mais recente sobre indicação do Mr. Scott Pruitt para administrar Agência de Proteção Ambiental (EPA) no EUA.

##### Fonte: https://www.theguardian.com/us-news/2016/dec/07/trump-scott-pruitt-environmental-protection-agency

####Keywords:

##### 'Donald Trump' (Novo presidente eleito nos Estados Unidos)
##### 'Scott Pruitt' (Indicação para ministerio do meio ambiente)
##### 'administrator of the Environmental' (ministerio do meio ambiente)
##### 'climate' (termo climatico)
##### 'fuel industry' (Indústria de combustíveis)
##### 'air pollution' (poluição do ar)
##### 'pollution' (poluição)
##### 'EPA' (Agência de Proteção Ambiental)
##### 'Barack Obama' (Ultimo presidente eleito nos Estados Unidos)

### Código Fonte

#### Coletando Dados do Twitter

In [11]:
# Importando os módulos Tweepy, Datetime e Json

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
from pymongo import MongoClient
import json


# Definições de acesso ao API Twitter

# Consumer Key
consumer_key = "mUZEhiqD4n2OwtWmEPDgpENwr"

# Consumer Secret
consumer_secret = "j9NcQ8bM5yPQC9tX9ALaTRHfufqDcjrvDVW28IKuJQ09OHIjci"

# Access Token
access_token = "809188036667965441-S9M5AzTINYLpJtZ3HTEOqSND4C4v0UX"

# Access Token Secret
access_token_secret = "bbY22BM8Lcd6ld0X84f2sGzMOh87Iq792xxtJP5zho83q"

# Criando as chaves de autenticação
auth = OAuthHandler(consumer_key, consumer_secret)

auth.set_access_token(access_token, access_token_secret)

# Criando uma classe para capturar os stream de dados do Twitter e armazenar no MongoDB
class MyListener(StreamListener):

    def on_data(self, dados):
        tweet = json.loads(dados)
        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        text = tweet["text"]
        obj = {"created_at":created_at,"id_str":id_str,"text":text,}
        tweetind = col.insert_one(obj).inserted_id
        print (obj)
        return True

# Criando o objeto pythonlistener
mylistener = MyListener()


# Criando o objeto stream
mystream = Stream(auth, listener = mylistener)

# Criando a conexão ao MongoDB
client = MongoClient('localhost', 27017)

# Criando o banco de dados Trabalho_Pratico
db = client.twitterdb

# Criando a collection "twitter"
col = db.tweets

# Criando uma lista de palavras chave para filtrar os Tweets
keywords = ['Donald Trump', 'Scott Pruitt', 'administrator of the Environmental', 'climate', 'fuel industry','air pollution','pollution', 'EPA', 'Barack Obama']

mystream.filter(track=keywords)

# Encerrando o stream de dados do Twitter
mystream.disconnect()

{'id_str': '812447975632936960', 'created_at': 'Sat Dec 24 00:01:06 +0000 2016', '_id': ObjectId('585dbac3efd71702f79d9ecb'), 'text': 'RT @dgardner: Too young to remember the Cold War? For a sense of what a nuclear arms race feels like, watch The Day After. https://t.co/b3d…'}
{'id_str': '812447976131989504', 'created_at': 'Sat Dec 24 00:01:06 +0000 2016', '_id': ObjectId('585dbac3efd71702f79d9ecc'), 'text': 'RT @veteranjames1: @bessbell Look bess! i just ordered the greatest Mug of Donald Trump from&gt;&gt; https://t.co/z5scukf5Pn'}
{'id_str': '812447975435804673', 'created_at': 'Sat Dec 24 00:01:06 +0000 2016', '_id': ObjectId('585dbac3efd71702f79d9ecd'), 'text': 'GO #scottpruitt https://t.co/1YJ5BQ6wMR'}
{'id_str': '812447975540748288', 'created_at': 'Sat Dec 24 00:01:06 +0000 2016', '_id': ObjectId('585dbac3efd71702f79d9ece'), 'text': 'RT @niggi: Hillary Clinton hat die Wahl mit knapp drei Millionen Stimmen Vorsprung vor Donald Trump verloren. https://t.co/vOHfySAWPP'}
{'id_str': '

#### Criando um Indice

In [6]:
%%bash

mongo
  
use twitterdb

db.tweets.createIndex({"text":1})





MongoDB shell version: 3.2.10
connecting to: test
switched to db twitterdb
{
	"createdCollectionAutomatically" : false,
	"numIndexesBefore" : 1,
	"numIndexesAfter" : 2,
	"ok" : 1
}
bye


#### Obter termos mais  frequentes (top 500+)

In [11]:
# Trabalhando com módulo Pandas para datasets e Importando o módulo Scikit-Learn

from pymongo import MongoClient
import pandas as pan
from sklearn.feature_extraction.text import CountVectorizer


# Conexão ao MongoDB
client = MongoClient('localhost', 27017)

# o banco de dados Trabalho_Pratico
db = client.twitterdb

# O collection "twitter"
col = db.tweets

# criando um dataset com dados retornados no collection Tweets

dataset = [{"created_at": item["created_at"],
            "text": item["text"],
           } for item in col.find()]

data = pan.DataFrame(dataset)

# o método CountVectorizer para criar uma matriz de documentos

cv = CountVectorizer()
count_matrix = cv.fit_transform(data.text)



# Contando o número de ocorrências das principais palavras no dataset

word_count = pan.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:500]





,word,count
0,https,790889
1,co,737585
2,rt,613886
3,trump,610153
4,donald,488682
5,the,408061
6,to,326033
7,of,220737
8,is,186201
9,and,161767


#### Volume x dia

In [19]:
%%bash

mongo
  
use twitterdb

var map = function() {
    var datetime = this._id.getTimestamp();

    var analise_por_dia = new Date(datetime.getFullYear(),
                                     datetime.getMonth(),
                                     datetime.getDate());
    emit(analise_por_dia, {count: 1});
}

var reduce = function(key, values) {
    var tot = 0;
    for(var x = 0; i < values.length; x++) { tot += values[x].count; }
    return {count: tot};
}

db.tweets_collection.mapReduce( map, reduce, { "out": "Volume_Dia" } );

db.Volume_Dia.find().limit(50).sort({"_id":-1})

# NOTA: TENTEI INUMERAS TENTATIVAS PARA SOLUCIONAR O ERRO. FAVOR COSNIDERAR A IMPLEMENTACAO DO CODIGO

MongoDB shell version: 3.2.10
connecting to: test
switched to db twitterdb
2016-12-23T23:26:59.842-0200 E QUERY    [thread1] Error: map reduce failed:{ "ok" : 0, "errmsg" : "ns doesn't exist" } :
_getErrorWithCode@src/mongo/shell/utils.js:25:13
DBCollection.prototype.mapReduce@src/mongo/shell/collection.js:1405:1
@(shell):1:1

2016-12-23T23:26:59.846-0200 E QUERY    [thread1] SyntaxError: illegal character @(shell):1:0

bye


#### Volume x hora do dia

In [20]:
%%bash

mongo
  
use twitterdb


var map = function() {
    var datetime = this._id.getTimestamp();

    var analise_por_hora  = new Date(datetime.getFullYear(),
                                     datetime.getMonth(),
                                     datetime.getDate(),
                                     datetime.getHours());
    emit(analise_por_hora , {count: 1});
}

var reduce = function(key, values) {
    var tot = 0;
    for(var x = 0; i < values.length; x++) { tot += values[x].count; }
    return {count: total};
}

db.tweets_collection.mapReduce( map, reduce, { "out": "Volume_Hora" } );

db.Volume_Hora.find().limit(10).sort({"_id":-1})


# NOTA: TENTEI INUMERAS TENTATIVAS PARA SOLUCIONAR O ERRO. FAVOR COSNIDERAR A IMPLEMENTACAO DO CODIGO

MongoDB shell version: 3.2.10
connecting to: test
switched to db twitterdb
2016-12-23T23:29:06.967-0200 E QUERY    [thread1] Error: map reduce failed:{ "ok" : 0, "errmsg" : "ns doesn't exist" } :
_getErrorWithCode@src/mongo/shell/utils.js:25:13
DBCollection.prototype.mapReduce@src/mongo/shell/collection.js:1405:1
@(shell):1:1

bye


### Referencias

#### Data Science Academy, Analisando stream de dados do Twitter com Mongdodb, URL: http://datascienceacademy.com.br/blog/2016/stream-de-dados-do-twitter-com-mongodb-pandas-e-scikit-learn/

#### Eduardo Santos, Extraindo dados de redes sociais com Python, URL: http://www.eduardosan.com/2015/06/09/extraindo-dados-de-redes-sociais-com-python/

#### GitHub Ana Paula Gomes, Repositorio codelab-analise-redes-sociais, URL: https://github.com/anapaulagomes/codelab-analise-redes-sociais/wiki/Coletando-Dados
